In [ ]:
import pandas as pd
import numpy as np 
import sklearn 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier,LinearRegression,LogisticRegression
from xgboost import XGBClassifier
from cuml import LogisticRegression,LinearRegression,Ridge,MBSGDClassifier
import cuml
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
"""text_clf=CatBoostClassifier(task_type="GPU",devices='0:1')
text_clf.fit(X_train_tfidf,y_train)
pred=text_clf.predict(X_test_tfidf)
score=sklearn.metrics.accuracy_score(y_test, pred)
print(score)"""

In [ ]:
train=pd.read_csv("../input/ruddit-jigsaw-dataset-combined-cleaned/toxic_train.csv")
X=train["processed"].rename({"processed":"text"})
y=train["isOffensive"].rename({"isOffensive":"toxic"})
to_score=pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
sample=pd.read_csv("../input/jigsaw-toxic-severity-rating/sample_submission.csv")
test=to_score.copy()

In [ ]:
import nltk.corpus
nltk.download("stopwords")
from nltk.corpus import stopwords
#df = pd.read_csv('...path/tmdb_5000_movies.csv')
stop_words = stopwords.words('english')
test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
stop_words = stopwords.words("english") + ["At", "v", "3"]
# apply the same code as above but assign
# a new column to see the differences
test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
"""text_clf = Pipeline([
     ('vect', CountVectorizer()),     
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss="modified_huber"))
])"""

In [ ]:
text_clf = Pipeline([
     ('vect', CountVectorizer()),     
     ('tfidf', TfidfTransformer()),
     ('clf', LGBMClassifier(device='gpu'))
])

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,shuffle=True)
text_clf.fit(x_train,y_train)

In [ ]:
pred=text_clf.predict(x_test)
score=sklearn.metrics.accuracy_score(y_test, pred)
print(score)

In [ ]:
"""0.94 without 
0.91 withs

0.9459 without 


0.91 without and withs

0.93 without and withs


"""

In [ ]:
"""count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_train)
X_test_counts=count_vect.transform(x_test)

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
"""

In [ ]:
"""0.937 sgd
0.91 mb
0.94 xgbbost
0.9477 LR
0.948 light"""

In [ ]:
p=text_clf.predict_proba(test.text)
sample.score=p[:,1]
sample.to_csv("submission.csv",index=False)